In [1]:
import requests
import requests_cache
requests_cache.install_cache('cache')
import requests_html
import math
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
from pprint import pprint
import re
import traceback
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_colwidth", None)

In [2]:
URL = "https://givealittle.co.nz/search/api?category=health&showClosed=True&page=cause&sort=newest"
first = requests.get(URL).json()
total = first["total"]
results = first["results"]
last_page = math.ceil(total / len(results))
print(f"Last page: {last_page}")
last_page = min(last_page, 1000)
for page in tqdm(range(2, last_page + 1)):
    page_results = requests.get(f"{URL}&p={page}").json()
    results.extend(page_results["results"])

Last page: 1201


  0%|          | 0/999 [00:00<?, ?it/s]

In [3]:
df = pd.DataFrame(results)
df.uri = "https://givealittle.co.nz" + df.uri
df.actionUri = "https://givealittle.co.nz" + df.actionUri
df

,title,pitch,hero,uri,amountRaised,goal,timeLeft,action,actionUri
0,Our little Aiden,Our little boy has the fight of his life ahead...,https://static.givealittle.co.nz/assets/hero/3...,https://givealittle.co.nz/cause/our-boy-aiden,11430.0,0.0,91 days to go,Donate,https://givealittle.co.nz/donate/cause/our-boy...
1,Dry July to raise funds for Effie,Dry July to raise funds for Effie,https://static.givealittle.co.nz/assets/hero/6...,https://givealittle.co.nz/cause/dry-july-to-ra...,320.0,1000.0,27 days to go,Donate,https://givealittle.co.nz/donate/cause/dry-jul...
2,Mother of 5 kids diagnosed with breast cancer,Family of 5 kids and 2 adults whose mum has St...,https://static.givealittle.co.nz/assets/hero/f...,https://givealittle.co.nz/cause/family-of-5s-m...,1045.0,0.0,90 days to go,Donate,https://givealittle.co.nz/donate/cause/family-...
3,Tyson’s 3rd fight with CANCER!,#fu*kcancer #tysons_cancer_journey,https://static.givealittle.co.nz/assets/hero/a...,https://givealittle.co.nz/cause/tysons-3rd-fig...,255.0,12000.0,90 days to go,Donate,https://givealittle.co.nz/donate/cause/tysons-...
4,Lunas Front Teeth,Help baby Luna fix her front teeth,https://static.givealittle.co.nz/assets/hero/9...,https://givealittle.co.nz/cause/lunas-front-teeth,35.0,9000.0,90 days to go,Donate,https://givealittle.co.nz/donate/cause/lunas-f...
...,...,...,...,...,...,...,...,...,...
9994,Amber and Daniel were twins born on 24th and 2...,Raising money for the little McSweeney family ...,https://static.givealittle.co.nz/assets/hero/a...,https://givealittle.co.nz/cause/hartleytwinsmi...,4834.0,0.0,closed,Read more,https://givealittle.co.nz/cause/hartleytwinsmi...
9995,"Tania Whanau, known by most as Tarnz, is a str...",Since February '16 Tarnz has undergone two ope...,https://static.givealittle.co.nz/assets/hero/6...,https://givealittle.co.nz/cause/fightbreastcan...,1550.0,0.0,closed,Read more,https://givealittle.co.nz/cause/fightbreastcan...
9996,Cardiology Fundraiser,Fundraising for patients & families affected b...,https://static.givealittle.co.nz/assets/hero/b...,https://givealittle.co.nz/cause/fund4aheart,915.0,0.0,closed,Read more,https://givealittle.co.nz/cause/fund4aheart
9997,"Help for Julia, while she is caring for Ian Ro...",Please support me so I'll be able to continue ...,https://static.givealittle.co.nz/assets/hero/5...,https://givealittle.co.nz/cause/help4julia,2645.0,0.0,closed,Read more,https://givealittle.co.nz/cause/help4julia


In [4]:
s = requests_html.HTMLSession()

In [5]:
def extract(row):
    uri = row.uri
    try:
        row = row.to_dict()
        html = s.get(uri).html
        if "This campaign started on" not in html.text:
            start = ""
            end = ""
        else:
            start, end = re.search(r"This campaign started on (.*) and (?:ends|ended) on (.*)\.", html.text).groups()
        question_link = html.find("a[href$=questions]", first=True)
        if question_link.text.strip() in ["Ask a question here", "Ask one here"]:
            n_questions = 0
        elif "One question has been asked already" in html.text:
            n_questions = 1
        else:
            n_questions = int(re.search("(\d+) questions have been asked already", html.text).group(1))
        use_of_funds = html.find("h4", containing="Use of funds", first=True)
        if use_of_funds:
            use_of_funds = use_of_funds.element.getnext().text_content()
        else:
            use_of_funds = ""
        whos_involved = html.find("h4", containing="Who's involved?", first=True)
        if whos_involved:
            whos_involved = whos_involved.element.getnext().text_content().strip()
        else:
            whos_involved = ""
        location = html.find("body > div > div > div.text-gray-600.pb-8.md\:pt-6 > div.w-full.max-w-6xl.mx-auto.px-6.space-y-2 > div.flex.pt-2.md\:pt-0 > div.w-full.md\:w-7\/12.lg\:w-2\/3.md\:shrink-0.space-y-6.divide-y.divide-gray-200 > div:nth-child(1) > p.mt-4", first=True)
        if location:
            location = location.text
        else:
            location = ""
        n_donors = html.find("body > div > div > div.text-gray-600.pb-8.md\:pt-6 > div.w-full.max-w-6xl.mx-auto.px-6.space-y-2 > div.flex.pt-2.md\:pt-0 > div.flex.flex-grow > div > div > div > div.font-serif > div.pt-1 > div > span", first=True)
        if n_donors:
            n_donors = int(n_donors.text)
        else:
            n_donors = ""
        row.update({
            "start": start,
            "end": end,
            "n_questions": n_questions,
            "location": location,
            "n_donors": n_donors,
            "description": html.find("div.break-words", first=True).text,
            "use_of_funds": use_of_funds,
            "whos_involved": whos_involved,
            "moderated": html.find("div.font-semibold")[-1].text
        })
        if "Latest update" in html.text:
            html = s.get(uri + "/updates").html
            updates = html.find("article")
            row.update({
                "n_updates": len(updates),
                "updates": "\n".join([update.text for update in updates])
            })
        if n_questions:
            html = s.get(uri + "/questions").html
            row["Q&A"] = html.find("body > div > div > div.w-full.max-w-6xl.mx-auto.px-6.py-6.text-gray-600.space-y-4 > div > div > div", first=True).text
        return row
    except Exception as e:
        print(f"{e} when trying to extract {uri}")
        print(traceback.format_exc())

df_extracted = pd.json_normalize(df.progress_apply(extract, axis=1))
df_extracted

  0%|          | 0/9999 [00:00<?, ?it/s]

,title,pitch,hero,uri,amountRaised,goal,timeLeft,action,actionUri,start,end,n_questions,location,n_donors,description,use_of_funds,whos_involved,moderated,n_updates,updates,Q&A
0,Our little Aiden,Our little boy has the fight of his life ahead...,https://static.givealittle.co.nz/assets/hero/3...,https://givealittle.co.nz/cause/our-boy-aiden,11430.0,0.0,91 days to go,Donate,https://givealittle.co.nz/donate/cause/our-boy...,3 Jul 2024,3 Oct 2024,0,Auckland,70,"Pete and Alyse’s little boy Aiden, aged 15 mon...",This will help aidens family get through the ...,Created by\r\nBecks Crabb \n\n\n\n\r\n P...,Page Moderated,1.0,Aiden list of treatments to date ❤️\n3 July 20...,NaN
1,Dry July to raise funds for Effie,Dry July to raise funds for Effie,https://static.givealittle.co.nz/assets/hero/6...,https://givealittle.co.nz/cause/dry-july-to-ra...,320.0,1000.0,27 days to go,Donate,https://givealittle.co.nz/donate/cause/dry-jul...,3 Jul 2024,31 Jul 2024,0,International,4,"Kia ora everyone,\nI'm Lucy, and this July, I'...",All money donated will go towards supporting R...,"Created by, and paying to a verified bank acco...",Page Moderated,NaN,NaN,NaN
2,Mother of 5 kids diagnosed with breast cancer,Family of 5 kids and 2 adults whose mum has St...,https://static.givealittle.co.nz/assets/hero/f...,https://givealittle.co.nz/cause/family-of-5s-m...,1045.0,0.0,90 days to go,Donate,https://givealittle.co.nz/donate/cause/family-...,3 Jul 2024,2 Oct 2024,0,"Taupo, Waikato",3,"My name is Kris. 44 years old, with 5 kids and...","Help with expenses, bills, travel and accomoda...","Created by, and paying to a verified bank acco...",Page Moderated,NaN,NaN,NaN
3,Tyson’s 3rd fight with CANCER!,#fu*kcancer #tysons_cancer_journey,https://static.givealittle.co.nz/assets/hero/a...,https://givealittle.co.nz/cause/tysons-3rd-fig...,255.0,12000.0,90 days to go,Donate,https://givealittle.co.nz/donate/cause/tysons-...,2 Jul 2024,2 Oct 2024,0,"Te Awamutu, Waikato",4,14 year old Tyson has been on cancer treatment...,Fuel lots of traveling 158kms between home and...,Created by\r\nMarc Mounsey \n\n\n\n\r\n ...,Page Moderated,NaN,NaN,NaN
4,Lunas Front Teeth,Help baby Luna fix her front teeth,https://static.givealittle.co.nz/assets/hero/9...,https://givealittle.co.nz/cause/lunas-front-teeth,35.0,9000.0,90 days to go,Donate,https://givealittle.co.nz/donate/cause/lunas-f...,2 Jul 2024,2 Oct 2024,0,Wellington,3,Little baby Luna has broken her front teeth an...,Paediatric dental specialist & replacement teeth,"Created by, and paying to a verified bank acco...",Page Moderated,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,Amber and Daniel were twins born on 24th and 2...,Raising money for the little McSweeney family ...,https://static.givealittle.co.nz/assets/hero/a...,https://givealittle.co.nz/cause/hartleytwinsmi...,4834.0,0.0,closed,Read more,https://givealittle.co.nz/cause/hartleytwinsmi...,2 May 2016,6 Jun 2016,0,Canterbury,122,PLEASE READ THE UPDATE TO THIS POST UNDER UPDA...,,"Created by, and paying to a verified bank acco...",Page Moderated,1.0,Attention all followers of our story\n31 May 2...,NaN
9995,"Tania Whanau, known by most as Tarnz, is a str...",Since February '16 Tarnz has undergone two ope...,https://static.givealittle.co.nz/assets/hero/6...,https://givealittle.co.nz/cause/fightbreastcan...,1550.0,0.0,closed,Read more,https://givealittle.co.nz/cause/fightbreastcan...,1 May 2016,22 Jul 2016,0,Hawke's Bay,34,"Tania Whanau, known by most as Tarnz, is a str...",,Created by\r\n \r\n Nancy Easthope...,Page Moderated,NaN,NaN,NaN
9996,Cardiology Fundraiser,Fundraising for patients & families affected b...,https://static.givealittle.co.nz/assets/hero/b...,https://givealittle.co.nz/cause/fund4aheart,915.0,0.0,closed,Read more,https://givealittle.co.nz/cause/fund4aheart,1 May 2016,15 Aug 2016,0,Waikato,9,We aim to provide clients with the most up-to-...,,"Created by, and paying to a verified bank acco...",Page 

In [6]:
df_extracted.to_csv("givealittle_health.csv", index=False)

In [7]:
df_extracted.to_excel("givealittle_health.xlsx", index=False)